In [1]:
import os
import numpy as np
from model import *
from util import *
from load import load_mnist_with_valid_set
import time 

n_epochs = 1000
learning_rate = 0.0002
batch_size = 128
image_shape = [28,28,1]
dim_z = 100
dim_W1 = 1024
dim_W2 = 128
dim_W3 = 64
dim_channel = 1

visualize_dim=196

data_dir = 'data/'

# only give the DCGAN 10% of training data
train_inds = np.loadtxt('train_inds.csv').astype(int)
trX, vaX, teX, trY, vaY, teY = load_mnist_with_valid_set(train_inds, percentage=0.1, data_dir=data_dir)

dcgan_model = DCGAN(
        batch_size=batch_size,
        image_shape=image_shape,
        dim_z=dim_z,
        dim_W1=dim_W1,
        dim_W2=dim_W2,
        dim_W3=dim_W3,
        )

Z_tf, Y_tf, image_tf, d_cost_tf, g_cost_tf, p_real, p_gen = dcgan_model.build_model()
sess = tf.InteractiveSession()

saver = tf.train.Saver()

saver = tf.train.import_meta_graph('models/mnist_gan.ckpt-500.meta')

saver.restore(sess, './models/mnist_gan.ckpt-500')


INFO:tensorflow:Restoring parameters from ./models/mnist_gan.ckpt-500


In [2]:
def discriminate_for_wb(data_to_be_discriminated, labels_to_be_discriminated, training_indicator):
    disc_results = np.zeros((len(data_to_be_discriminated),2))
    
    data_to_be_discriminated = data_to_be_discriminated.reshape( [-1, 28, 28, 1]) / 255
    
    disc_results[:,1] = training_indicator
    
    for iteration in range(len(data_to_be_discriminated) // batch_size):
        start = iteration*batch_size
        end = (iteration+1)*batch_size
        
        ind = np.arange(start,end)
        Xs = tf.to_float(data_to_be_discriminated[ind].reshape( [-1, 28, 28, 1]) / 255)
        Ys = tf.to_float(OneHot(labels_to_be_discriminated[ind]))
        disc_results[ind, 0] = np.reshape(sess.run(dcgan_model.discriminate(Xs ,Ys)),(batch_size,))
    
    # fill last few elements
    ind = np.arange(len(data_to_be_discriminated)-batch_size,len(data_to_be_discriminated))
    Xs = tf.to_float(data_to_be_discriminated[ind].reshape( [-1, 28, 28, 1]) / 255)
    Ys = tf.to_float(OneHot(labels_to_be_discriminated[ind]))
    disc_results[ind, 0] = np.reshape(sess.run(dcgan_model.discriminate(Xs ,Ys)),(batch_size,))
    
    return disc_results

def wb_attack_sample(disc_results_train, disc_results_validate):
    results = np.concatenate((disc_results_train,disc_results_validate))
    np.random.shuffle(results)
    results = results[results[:,0].argsort()]

    return results[-len(disc_results_train):,1].mean()




In [3]:
trX_inds = np.arange(len(trX))
np.random.shuffle(trX_inds)
trX_inds = trX_inds[0:100]

vaX_inds = np.arange(len(trX))
np.random.shuffle(vaX_inds)
vaX_inds = vaX_inds[0:100]

In [4]:
disc_results_train = discriminate_for_wb(trX[trX_inds],trY[trX_inds],1)
disc_results_validate = discriminate_for_wb(vaX[vaX_inds],vaY[vaX_inds],0)

In [11]:
results = np.concatenate((disc_results_train,disc_results_validate))
np.random.shuffle(results)
results

array([[ 2.37787843e+00,  1.00000000e+00],
       [ 3.45189989e-01,  1.00000000e+00],
       [ 1.91918659e+00,  0.00000000e+00],
       [-1.20474282e-03,  1.00000000e+00],
       [-2.87329972e-01,  0.00000000e+00],
       [-1.25389308e-01,  0.00000000e+00],
       [-7.78145716e-03,  1.00000000e+00],
       [ 4.92255315e-02,  0.00000000e+00],
       [ 5.34343839e-01,  0.00000000e+00],
       [-1.64367884e-01,  0.00000000e+00],
       [-1.20569810e-01,  1.00000000e+00],
       [ 3.68106246e-01,  1.00000000e+00],
       [ 1.84185326e-01,  1.00000000e+00],
       [ 2.36753511e+00,  1.00000000e+00],
       [-2.86532938e-01,  0.00000000e+00],
       [ 2.64970958e-01,  0.00000000e+00],
       [ 6.61509782e-02,  0.00000000e+00],
       [ 5.65841198e-01,  0.00000000e+00],
       [ 5.55119455e-01,  1.00000000e+00],
       [ 1.23847499e-01,  1.00000000e+00],
       [ 3.53408337e-01,  1.00000000e+00],
       [-2.84854352e-01,  0.00000000e+00],
       [-2.50900209e-01,  1.00000000e+00],
       [-2.

In [14]:
results = results[results[:,0].argsort()]
results

array([[-2.88460195e-01,  0.00000000e+00],
       [-2.88225710e-01,  0.00000000e+00],
       [-2.87329972e-01,  0.00000000e+00],
       [-2.86532938e-01,  0.00000000e+00],
       [-2.86346734e-01,  0.00000000e+00],
       [-2.86009490e-01,  0.00000000e+00],
       [-2.85584271e-01,  0.00000000e+00],
       [-2.85374343e-01,  0.00000000e+00],
       [-2.84854352e-01,  0.00000000e+00],
       [-2.84653962e-01,  0.00000000e+00],
       [-2.82127440e-01,  1.00000000e+00],
       [-2.81627357e-01,  1.00000000e+00],
       [-2.80916631e-01,  1.00000000e+00],
       [-2.80457556e-01,  1.00000000e+00],
       [-2.80289590e-01,  1.00000000e+00],
       [-2.80098736e-01,  1.00000000e+00],
       [-2.79702187e-01,  1.00000000e+00],
       [-2.79312730e-01,  1.00000000e+00],
       [-2.78788984e-01,  1.00000000e+00],
       [-2.77735531e-01,  1.00000000e+00],
       [-2.52530396e-01,  0.00000000e+00],
       [-2.51021206e-01,  0.00000000e+00],
       [-2.50900209e-01,  1.00000000e+00],
       [-2.

In [21]:
results[-len(disc_results_train):]

array([[0.15124771, 0.        ],
       [0.15233879, 1.        ],
       [0.15253504, 1.        ],
       [0.16233855, 1.        ],
       [0.16251215, 0.        ],
       [0.17179781, 0.        ],
       [0.1718522 , 0.        ],
       [0.17238942, 0.        ],
       [0.17335674, 0.        ],
       [0.17719401, 1.        ],
       [0.18159388, 1.        ],
       [0.18386501, 1.        ],
       [0.18418533, 1.        ],
       [0.18605012, 0.        ],
       [0.18728194, 1.        ],
       [0.20548144, 0.        ],
       [0.21458362, 1.        ],
       [0.24239409, 0.        ],
       [0.25077665, 0.        ],
       [0.25249925, 0.        ],
       [0.25268811, 0.        ],
       [0.25887397, 0.        ],
       [0.26089534, 0.        ],
       [0.26497096, 0.        ],
       [0.26731145, 0.        ],
       [0.28031719, 0.        ],
       [0.28544828, 0.        ],
       [0.28722492, 1.        ],
       [0.28983426, 1.        ],
       [0.29707551, 1.        ],
       [0.

In [39]:
def generate_samples_for_digits(sample_size=100):
    
    Z_np_sample = np.random.randn(sample_size, dim_z)
    
    digits = np.zeros((sample_size,)).astype(int)
    for i in range(len(digits)):
        digits[i] = i%10
    Y_np_sample = OneHot( digits)

    Z_tf_sample, Y_tf_sample, image_tf_sample = dcgan_model.samples_generator(batch_size=sample_size)

    generated_samples = sess.run(
                    image_tf_sample,
                    feed_dict={
                        Z_tf_sample:Z_np_sample,
                        Y_tf_sample:Y_np_sample
                        })
    generated_samples = (generated_samples + 1.)/2.
    return generated_samples

def save_image(data, path):
    img = np.zeros((28, 28, 3))
    img[0:28, 0:28, :] = data.reshape( [28, 28, 1])
    scipy.misc.imsave('./'+path+'.jpg', img)

In [36]:
[3+10*i for i in range(1000)]

[3,
 13,
 23,
 33,
 43,
 53,
 63,
 73,
 83,
 93,
 103,
 113,
 123,
 133,
 143,
 153,
 163,
 173,
 183,
 193,
 203,
 213,
 223,
 233,
 243,
 253,
 263,
 273,
 283,
 293,
 303,
 313,
 323,
 333,
 343,
 353,
 363,
 373,
 383,
 393,
 403,
 413,
 423,
 433,
 443,
 453,
 463,
 473,
 483,
 493,
 503,
 513,
 523,
 533,
 543,
 553,
 563,
 573,
 583,
 593,
 603,
 613,
 623,
 633,
 643,
 653,
 663,
 673,
 683,
 693,
 703,
 713,
 723,
 733,
 743,
 753,
 763,
 773,
 783,
 793,
 803,
 813,
 823,
 833,
 843,
 853,
 863,
 873,
 883,
 893,
 903,
 913,
 923,
 933,
 943,
 953,
 963,
 973,
 983,
 993,
 1003,
 1013,
 1023,
 1033,
 1043,
 1053,
 1063,
 1073,
 1083,
 1093,
 1103,
 1113,
 1123,
 1133,
 1143,
 1153,
 1163,
 1173,
 1183,
 1193,
 1203,
 1213,
 1223,
 1233,
 1243,
 1253,
 1263,
 1273,
 1283,
 1293,
 1303,
 1313,
 1323,
 1333,
 1343,
 1353,
 1363,
 1373,
 1383,
 1393,
 1403,
 1413,
 1423,
 1433,
 1443,
 1453,
 1463,
 1473,
 1483,
 1493,
 1503,
 1513,
 1523,
 1533,
 1543,
 1553,
 1563,
 1573,
 1583

In [53]:
generated_samples = generate_samples_for_digits(100)
# all sevens
generated_samples[[7+10*i for i in range(100//10)]]

c:\users\d065042\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


In [49]:
save_image(generate_samples_for_digits(100)[99],'generated')

c:\users\d065042\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


In [25]:
import cv2
import numpy as np
import scipy
from scipy.misc import imread
import random
import os
import matplotlib.pyplot as plt

# Feature extractor
def extract_features(image, vector_size=10):
    try:
        # Using KAZE, cause SIFT, ORB and other was moved to additional module
        # which is adding addtional pain during install
        alg = cv2.KAZE_create()
        # Dinding image keypoints
        kps = alg.detect(image)
        # Getting first 32 of them. 
        # Number of keypoints is varies depend on image size and color pallet
        # Sorting them based on keypoint response value(bigger is better)
        kps = sorted(kps, key=lambda x: -x.response)[:vector_size]
        kps = sorted(kps, key=lambda x: -x.pt[0])
        # computing descriptors vector
        kps, dsc = alg.compute(image, kps)
        print(kps[0].pt[0])
        print(dsc)
        needed_size = (vector_size * 64)
        
        if dsc is None:
            dsc = np.zeros(needed_size)
        else:
            dsc = dsc.flatten()
            if dsc.size < needed_size:
                dsc = np.concatenate([dsc, np.zeros(needed_size - dsc.size)])
    except cv2.error as e:
        return e

    return dsc

def generate_batch_features(samples, uint8):
    features_matrix = np.zeros((len(samples),2048))

    for i in range(len(samples)):
        if uint8:
            features_matrix[i] = extract_features(samples[i].reshape( [28, 28, 1]).astype(np.uint8))
        else:
            features_matrix[i] = extract_features(samples[i].reshape( [28, 28, 1]))
    
    return features_matrix

In [26]:
feature_matrix_trX = generate_batch_features([trX[0]], True)
feature_matrix_trX

19.127899169921875
[[-4.51417230e-02 -4.53914627e-02  4.56688069e-02  4.54313047e-02
  -7.77438432e-02 -9.84757394e-02  1.21858381e-01  1.42419860e-01
  -8.37028306e-03  1.10414922e-02  4.01104055e-02  5.95381334e-02
  -5.22983400e-03 -3.84625327e-03  1.11523559e-02  1.27210375e-02
  -1.54165132e-02  8.39643460e-03  1.64364353e-02  9.29982308e-03
   1.51622714e-02  2.40484136e-03  1.23451158e-01  7.09307492e-02
  -2.37650182e-02 -5.03847608e-03  9.47486311e-02  8.55843946e-02
  -1.21921068e-02 -1.76730007e-03  1.49500379e-02  8.26271903e-03
  -4.01694357e-01  2.98757195e-01  4.01749730e-01  2.98757195e-01
  -1.72187552e-01  1.42230004e-01  2.01076284e-01  1.81144908e-01
   5.15178451e-03 -2.32352186e-02  3.68433483e-02  4.65898886e-02
  -9.35022440e-03  6.15846226e-03  1.71023030e-02  1.32530630e-02
  -2.68129170e-01  2.41965324e-01  2.68129170e-01  2.41965324e-01
  -4.04440425e-02  3.93306389e-02  4.04666848e-02  3.93561423e-02
   2.53325416e-05  3.20467848e-06  8.01327266e-03  6.4331

ValueError: could not broadcast input array from shape (640) into shape (2048)

In [2]:
trX_inds = np.arange(len(trX))
np.random.shuffle(trX_inds)
trX_inds = trX_inds[0:100]

In [5]:
np.where(trY[trX_inds] == 3)

(array([ 0, 16, 18, 27, 72, 75, 94, 97], dtype=int64),)

In [9]:
generated_samples = generate_samples(30)

print(extract_features(generated_samples[1].reshape( [28, 28, 1])))
print(extract_features(trX[1].reshape( [28, 28, 1]).astype(np.uint8)))

[-0.16380988 -0.10773352  0.17749077 ...  0.          0.
  0.        ]
[ 0.01810544 -0.00857279  0.11762071 ...  0.          0.
  0.        ]


In [29]:
#index = 10

#features_searched = extract_features(trX[index].reshape( [28, 28, 1]).astype(np.uint8))

mc_sample_size = 100
instance_no = 2

## 

trX_inds = np.arange(len(trX))
np.random.shuffle(trX_inds)
trX_inds = trX_inds[0:100]
np.save(str(instance_no)+'_trX_inds', trX_inds)
feature_matrix_trX = generate_batch_features(trX[trX_inds], True)
np.save('feature_matrix_trX', feature_matrix_trX)

vaX_inds = np.arange(len(trX))
np.random.shuffle(vaX_inds)
vaX_inds = vaX_inds[0:100]
np.save(str(instance_no)+'_vaX_inds', vaX_inds)
feature_matrix_vaX = generate_batch_features(vaX[vaX_inds], True)
np.save('feature_matrix_vaX', feature_matrix_vaX)

##generated samples

generated_samples = generate_samples(mc_sample_size)
feature_matrix_generated = generate_batch_features(generated_samples, False)
np.save('feature_matrix_generated', feature_matrix_generated)

In [34]:
scipy.spatial.distance.cdist(feature_matrix_trX, feature_matrix_generated, 'euclidean')
scipy.spatial.distance.cdist(feature_matrix_vaX, feature_matrix_generated, 'euclidean')

array([[2.85466514, 3.75263614, 2.88156082, ..., 2.17110535, 2.53716035,
        2.8168601 ],
       [3.10909256, 3.72313859, 3.0542255 , ..., 2.22968089, 2.35756927,
        2.61772376],
       [3.34180393, 3.74103888, 3.06553933, ..., 1.98303888, 2.83645114,
        2.53437722],
       ...,
       [3.30032823, 3.60536838, 2.98794469, ..., 1.98802071, 2.70027927,
        2.55341159],
       [3.73397239, 4.04506927, 3.89815577, ..., 3.75157609, 3.94104117,
        3.87381375],
       [3.4866127 , 3.95660389, 3.39228735, ..., 2.8977535 , 3.58038473,
        2.51251529]])

In [26]:
distances = scipy.spatial.distance.cdist([features_searched], features_gen, 'euclidean')

print(distances)
min_indexes = distances[0][:].argsort()
save_image(trX[index], 'example_knn/trX0')
save_image(generated_samples[min_indexes[0]], 'example_knn/candidate0')
save_image(generated_samples[min_indexes[1]], 'example_knn/candidate1')
save_image(generated_samples[min_indexes[2]], 'example_knn/candidate2')
save_image(generated_samples[min_indexes[3]], 'example_knn/candidate3')
save_image(generated_samples[min_indexes[4]], 'example_knn/candidate4')

[[3.34457629 3.5908696  3.64392676 3.51305749 3.5012165  3.89595542
  3.4930585  3.86500265 3.63695285 3.50506577 4.11007743 3.35529082
  4.40636965 3.69387826 3.80148412 3.91225602 3.99476113 3.88188571
  3.68395102 3.59457681]]


c:\users\d065042\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


In [211]:
import time

def print_elapsed_time():
    end_time = int(time.time())
    d = divmod(end_time-start_time,86400)  # days
    h = divmod(d[1],3600)  # hours
    m = divmod(h[1],60)  # minutes
    s = m[1]  # seconds

    print('Elapsed Time: %d days, %d hours, %d minutes, %d seconds' % (d[0],h[0],m[0],s))

In [217]:
def calculate_results_matrices(distances_real_vs_sample,distances_real_vs_train, d_min=0.1):

    results_sample = np.zeros((len(distances_real_vs_sample),3))
    for i in range(len(results_sample)):
        # indicate that dataset is a sample
        results_sample[i][0] = 0
        
        integral_approx = 0
        integral_approx_eps = 0
        for eps in distances_real_vs_sample[i]:
            if eps < d_min:
                integral_approx = integral_approx + (-np.log(eps/d_min))
                integral_approx_eps = integral_approx_eps + 1
        integral_approx = integral_approx/len(distances_real_vs_sample[0])
        integral_approx_eps = integral_approx_eps/len(distances_real_vs_sample[0])

        results_sample[i][1] = integral_approx
        results_sample[i][2] = integral_approx_eps

    results_train = np.zeros((len(distances_real_vs_train),3))
    for i in range(len(results_train)):
        # indicate that dataset is a training data set
        results_train[i][0] = 1
        
        integral_approx = 0
        integral_approx_eps = 0
        for eps in distances_real_vs_train[i]:
            if eps < d_min:
                integral_approx = integral_approx + (-np.log(eps/d_min))
                integral_approx_eps = integral_approx_eps + 1
        integral_approx = integral_approx/len(distances_real_vs_train[0])
        integral_approx_eps = integral_approx_eps/len(distances_real_vs_train[0])

        results_train[i][1] = integral_approx
        results_train[i][2] = integral_approx_eps
        
    return results_sample,results_train

In [10]:
mc_sample_size = 40

euclidean_trX = np.reshape(trX, (len(trX),784,))
trX_inds = np.arange(len(trX))
np.random.shuffle(trX_inds)
trX_inds = trX_inds[0:100]
np.save(str(1)+'_trX_inds', trX_inds)
euclidean_trX = euclidean_trX[trX_inds]

euclidean_vaX = np.reshape(vaX, (len(vaX),784,))
vaX_inds = np.arange(len(trX))
np.random.shuffle(vaX_inds)
vaX_inds = vaX_inds[0:100]
np.save(str(1)+'_vaX_inds', vaX_inds)
euclidean_vaX = euclidean_vaX[vaX_inds]

start_time = int(time.time())

def print_elapsed_time():
    end_time = int(time.time())
    d = divmod(end_time-start_time,86400)  # days
    h = divmod(d[1],3600)  # hours
    m = divmod(h[1],60)  # minutes
    s = m[1]  # seconds

    print('Elapsed Time: %d days, %d hours, %d minutes, %d seconds' % (d[0],h[0],m[0],s))

for i in range(2):

    print('Generating Samples %d'%(i))

    euclidean_generated_samples = generate_samples(mc_sample_size)

    print('Reshaping... %d'%(i))

    euclidean_generated_samples = euclidean_generated_samples - euclidean_generated_samples.min()
    euclidean_generated_samples = euclidean_generated_samples*255/euclidean_generated_samples.max()
    euclidean_generated_samples = np.reshape(euclidean_generated_samples, (len(euclidean_generated_samples),784,))

    print('Calculating Distances... %d'%(i))

    distances_trX_partial = scipy.spatial.distance.cdist(euclidean_trX, euclidean_generated_samples, 'euclidean')
    distances_vaX_partial = scipy.spatial.distance.cdist(euclidean_vaX, euclidean_generated_samples, 'euclidean')
    
    if i == 0:
        distances_trX = distances_trX_partial
        distances_vaX = distances_vaX_partial
    else:
        distances_trX = np.column_stack((distances_trX, distances_trX_partial))
        distances_vaX = np.column_stack((distances_vaX, distances_vaX_partial))
    
    print_elapsed_time()

Generating Samples 0
Reshaping... 0
Calculating Distances... 0
Elapsed Time: 0 days, 0 hours, 0 minutes, 0 seconds
Generating Samples 1
Reshaping... 1
Calculating Distances... 1
Elapsed Time: 0 days, 0 hours, 0 minutes, 1 seconds


In [24]:
start_time = int(time.time())
#distances = scipy.spatial.distance.cdist([trX[0]], generated_samples, 'euclidean')
euclidean_generated_samples = generate_samples(1000)
euclidean_generated_samples = euclidean_generated_samples - euclidean_generated_samples.min()
euclidean_generated_samples = euclidean_generated_samples*255/euclidean_generated_samples.max()
euclidean_generated_samples = np.reshape(euclidean_generated_samples, (len(euclidean_generated_samples),784,))
euclidean_trX = np.reshape(trX, (len(trX),784,))
euclidean_vaX = np.reshape(vaX, (len(vaX),784,))

distances_trX = scipy.spatial.distance.cdist(euclidean_trX, euclidean_generated_samples, 'euclidean')
distances_vaX = scipy.spatial.distance.cdist(euclidean_vaX, euclidean_generated_samples, 'euclidean')
print_elapsed_time()

NameError: name 'generate_samples' is not defined

In [275]:
np.save('distances_trX', distances_trX)
np.save('distances_vaX', distances_vaX)

In [15]:
results_sample = np.load('1_results_sample_0.1.npy')
results_train = np.load('1_results_train_0.1.npy')

In [25]:
trX_inds = np.arange(len(trX))
np.random.shuffle(trX_inds)
trX_inds = trX_inds[0:100]
np.save('trX_inds', trX_inds)

vaX_inds = np.arange(len(trX))
np.random.shuffle(vaX_inds)
vaX_inds = vaX_inds[0:100]
np.save('vaX_inds', vaX_inds)


In [269]:
start_time = int(time.time())

d_min = np.percentile(np.concatenate((distances_trX,distances_vaX)),10)
results_sample,results_train = calculate_results_matrices(distances_vaX, distances_trX,d_min)

np.save('results_sample_10', results_sample)
np.save('results_train_10', results_train)

d_min = np.percentile(np.concatenate((distances_trX,distances_vaX)),1)
results_sample,results_train = calculate_results_matrices(distances_vaX, distances_trX,d_min)

np.save('results_sample_1', results_sample)
np.save('results_train_1', results_train)

d_min = np.percentile(np.concatenate((distances_trX,distances_vaX)),0.1)
results_sample,results_train = calculate_results_matrices(distances_vaX, distances_trX,d_min)

np.save('results_sample_0.1', results_sample)
np.save('results_train_0.1', results_train)

print_elapsed_time()

In [277]:
#results = np.concatenate((results_sample, results_train))
#np.random.shuffle(results)
len(results_sample)

54000

In [272]:
results = np.concatenate((results_sample, results_train))
results = results[results[:,2].argsort()][:,0]
print('mc_attack: '+results_train[-len(trX)*offset:-offset].mean())
    
print_elapsed_time()

0.10118353058843141
0.10168361393565595
0.09790520642329277
0.09914152358726454
0.09908318053008834
0.0998221925876535
0.10061200676303193
0.1006209368228038
0.1010909225611676
0.10001666944490749
Elapsed Time: 0 days, 0 hours, 7 minutes, 40 seconds


In [253]:
len(trX)/len(vaX)

0.1111111111111111

In [209]:
index = 20
save_image(trX[index], 'example_knn/trX0')
min_indexes = distances[index].argsort()
save_image(generated_samples[min_indexes[0]], 'example_knn/candidate0')
save_image(generated_samples[min_indexes[1]], 'example_knn/candidate1')
save_image(generated_samples[min_indexes[2]], 'example_knn/candidate2')
save_image(generated_samples[min_indexes[3]], 'example_knn/candidate3')
save_image(generated_samples[min_indexes[4]], 'example_knn/candidate4')

c:\users\d065042\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


In [199]:
import scipy
#distances = scipy.spatial.distance.cdist([features_searched], features_gen, 'cosine')
distances = scipy.spatial.distance.cdist([features_searched], features_gen, 'euclidean')
min_indexes = distances[0].argsort()

save_image(generated_samples[min_indexes[0]], 'example_knn/candidate0')
save_image(generated_samples[min_indexes[1]], 'example_knn/candidate1')
save_image(generated_samples[min_indexes[2]], 'example_knn/candidate2')
save_image(generated_samples[min_indexes[3]], 'example_knn/candidate3')
save_image(generated_samples[min_indexes[4]], 'example_knn/candidate4')

c:\users\d065042\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


In [141]:
distances[0][min_indexes[0:4]]

array([1.68590236, 1.71218312, 1.72991279, 1.82317544])

In [83]:
img = np.zeros((28, 28, 3))
img[0:h, 0:w, :] = vaX[1].reshape( [28, 28, 1])
img = img.astype(np.uint8)

vector_size = 32
try:
    # Using KAZE, cause SIFT, ORB and other was moved to additional module
    # which is adding addtional pain during install
    alg = cv2.KAZE_create()
    # Dinding image keypoints
    kps = alg.detect(img)
    # Getting first 32 of them. 
    # Number of keypoints is varies depend on image size and color pallet
    # Sorting them based on keypoint response value(bigger is better)
    kps = sorted(kps, key=lambda x: -x.response)[:vector_size]
    # computing descriptors vector
    kps, dsc = alg.compute(img, kps)
    # Flatten all of them in one big vector - our feature vector
    dsc = dsc.flatten()
    # Making descriptor of same size
    # Descriptor vector size is 64
    needed_size = (vector_size * 64)
    if dsc.size < needed_size:
        # if we have less the 32 descriptors then just adding zeros at the
        # end of our feature vector
        dsc = np.concatenate([dsc, np.zeros(needed_size - dsc.size)])
except cv2.error as e:
    print(e)
    
dsc

array([-0.32678679, -0.36212644,  0.32678679, ...,  0.        ,
        0.        ,  0.        ])

In [68]:
np.array(img).shape



(28, 28, 3)

In [71]:
type(image[0][0][0])

numpy.uint8